## UAV fleet Simulation 

This is for testing propouses

In [ ]:
import os
import subprocess
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from helpers.visualization.gazebo_world import update_world
from helpers.change_coordinates import local2global,heading_to_yaw,find_spawns
from helpers.visualization.plots import plot_3d_interactive

from plan import Plan,State

Kill all related process

In [ ]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [ ]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/runway.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

## Choose Simulator

In [ ]:
simulator ='gazebo' #'QGroundControl' # 

## Create Plans

In [ ]:
offset = (0, 0, 0, 0) #east, north, up, heading
    
path = Plan.create_square_path(side_len=7,alt= 5)
home=np.array(offset[:3])

global_path=local2global(np.array(path), home,pairwise=True) 

plan = Plan.basic(wps=path,alt=5,wp_margin=0.5,verbose=1)


markers = {'waypoints': {'pos':global_path,'color':'blue'},
           'home':      {'pos':[home],'color':'orange'}}


if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,0)
    spawn=find_spawns(home_position, offset)

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)



Launch Simulator

In [ ]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    east, north, up, heading = offset
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading))]
    world_path = os.path.expanduser(gazebo_word_path)
    del markers['home']
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

Launch Vehicles

In [ ]:
vehicle_cmd = f"python3 {ardupilot_vehicle_path} -v ArduCopter -I0 --sysid 1 --no-rebuild"
if simulator == 'QGroundControl':
    vehicle_cmd += f" --custom-location={spawn}"
elif simulator == 'gazebo':
    vehicle_cmd += " -f gazebo-iris"
subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

Create UAV

In [ ]:
uav = VehicleLogic(sys_id=1,
                    home=home,
                    plan= plan)

# uav = VehicleLogic(sys_id=1,
#                     home=home)

## Execute Plan

In [ ]:
while uav.plan.state!=State.DONE:
    uav.act_plan()

## Kill all related process

In [ ]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")